In [2]:
search_for = 135
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0212705135345459
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 723027151
type: [1, 1, 1, 1, 135] 135
cases of this type: 332150625
100000 28.8879800183309
200000 51.05191163379875
300000 50.33630408329616
400000 51.93585212174955
500000 57.286237307072774
600000 67.95377171406425
700000 74.7756601793577
800000 79.5777629994292
900000 81.08152876902915
1000000 75.3106191120129
1100000 74.21678548294955
1200000 66.53911176512226
1300000 24.501426295290322
1400000 69.48504080172481
1500000 67.11512147644721
1600000 69.4335221049863
1700000 69.44370517989901
1800000 74.34411035607641
1900000 70.67249094874707
2000000 69.53735641762205
2100000 70.16846888978498
2200000 68.49049592381519
2300000 69.57243319085718
2400000 72.4511967291633
2500000 21.902028854471528
2600000 68.35252502687645
2700000 87.6494392864014
2800000 92.01600595059936
2900000 85.19340713463535
3000000 88.98964371888343
310000

30100000 75.80380144305467
30200000 69.87445885377286
30300000 68.63904993505258
30400000 73.83181336448555
30500000 68.8836197083489
30600000 68.42796244233372
30700000 68.08335623299682
30800000 15.732029652154678
30900000 71.59746679516157
31000000 65.47246027940811
31100000 68.33324544065854
31200000 70.37628518614112
31300000 68.63509958375255
31400000 67.52952305557471
31500000 77.25384544286125
31600000 72.60183307359087
31700000 70.72893443226472
31800000 25.689391174922505
31900000 75.5522960336419
32000000 70.29666556804973
32100000 77.36353187886944
32200000 70.36074831967944
32300000 30.494503193176563
32400000 78.55873835056758
32500000 74.24933319141059
32600000 74.8069981616825
32700000 71.65625367422176
32800000 75.71065412099749
32900000 71.57146227024833
33000000 72.76117801373299
33100000 66.34180103204697
33200000 53.75957766244134
33300000 36.08135840196808
33400000 69.44118484733747
33500000 68.41405306587482
33600000 75.00593635378432
33700000 78.32328684188333
3

60500000 64.22055694159057
60600000 65.81770076977932
60700000 64.52444424391057
60800000 66.22511323199419
60900000 66.08457298133442
61000000 70.34097186927168
61100000 27.78643072863915
61200000 79.41764829208405
61300000 70.02747494199339
61400000 69.19151601419661
61500000 70.52427932716252
61600000 73.97734374495536
61700000 72.10905319198167
61800000 75.22353683703186
61900000 71.25017258178605
62000000 34.71199215501585
62100000 64.92298019241028
62200000 69.683350205717
62300000 71.25467979769392
62400000 68.75326132387005
62500000 67.2673383442502
62600000 66.33030532688475
62700000 64.70330477670132
62800000 19.00954203770191
62900000 68.22826780885313
63000000 64.80075623312139
63100000 69.64391185093282
63200000 63.209944234057524
63300000 68.94156921074358
63400000 68.72310864022835
63500000 51.708964026557794
63600000 51.005285990367646
63700000 72.35020524104313
63800000 75.75343692268713
63900000 72.01695984603568
64000000 75.0867776375096
64100000 72.50889687404349
64

90800000 68.58480975947786
90900000 65.02316276285127
91000000 62.291175858476976
91100000 64.32898662142807
91200000 64.26113119526828
91300000 63.46477945338761
91400000 66.01332714892601
91500000 58.24036389043656
91600000 67.40269738036315
91700000 46.30729109656407
91800000 37.4503041762338
91900000 55.33342545092733
92000000 61.6604550903552
92100000 56.977073008671105
92200000 62.57603238865613
92300000 19.07551986344066
92400000 58.99219042944645
92500000 56.194731456516784
92600000 58.66109927146163
92700000 59.01812014086613
92800000 45.21133556093173
92900000 38.03277460323809
93000000 63.25669797970265
93100000 59.415374190954815
93200000 64.76679778180616
93300000 62.41397535564188
93400000 64.64409688658269
93500000 66.07102591473888
93600000 66.06232470886134
93700000 68.76913756622531
93800000 63.24979624135917
93900000 63.31814112764745
94000000 60.47852825333396
94100000 61.895167000968286
94200000 39.3378594575046
94300000 41.44616844268445
94400000 58.44901938327156

120300000 63.86346697643233
120400000 64.64655456601544
120500000 63.932374655852
120600000 73.73182225914357
120700000 65.95287051967973
120800000 64.65913844732462
120900000 66.63650597828612
121000000 64.58859027858539
121100000 63.81093106248698
121200000 58.772365660480865
121300000 60.729198648891526
121400000 62.429312513877946
121500000 22.02301927179723
121600000 61.078178332467026
121700000 58.75386241117153
121800000 31.536326235281212
121900000 45.93834182493919
122000000 59.57260812032227
122100000 46.46420003047945
122200000 42.74992868067156
122300000 58.09172751717081
122400000 59.980885860493835
122500000 58.38919123722038
122600000 62.0271320435513
122700000 67.78974112216477
122800000 64.52833631971308
122900000 63.87204344232467
123000000 63.816403966898235
123100000 66.79267563932679
123200000 66.04865120556448
123300000 64.38588997299695
123400000 64.72488227242766
123500000 71.89213151193275
123600000 66.78137892447793
123700000 59.758221309367
123800000 57.33273

...

639800000 1.83459380997796
639900000 1.8703952835642876
640000000 1.8125284401398423
640100000 1.98242053664045
640200000 2.095443770465556
640300000 2.061230502601574
640400000 1.9054808701807189
640500000 1.846669305700291
640600000 1.8339961921113928
640700000 1.7227618239951172
640800000 1.767557007336526
640900000 2.0122952717555043
641000000 2.231040844468194
641100000 2.0523416817615168
641200000 1.8523771176631632
641300000 1.7648367635082733
641400000 1.7246063495186885
641500000 1.6123874061879397
641600000 1.7282307272080937
641700000 1.8754335592880056
641800000 2.0900653956365165
641900000 2.0987184546498
642000000 1.9476505575176555
642100000 1.7548605915211437
642200000 1.6475023114159677
642300000 1.5508989284453238
642400000 1.5860095089019877
642500000 1.6952757894836183
642600000 1.8415825914479316
642700000 2.035170505857509
642800000 1.8946617736182234
642900000 1.7692790996279868
643000000 1.681041433282912
643100000 1.530788315707045
643200000 1.5178189818267394


667500000 2.231326192596315
667600000 2.295319986708218
667700000 2.275812362835712
best so far: 0
type: [1, 1, 5, 3, 9] 135
cases of this type: 4428675
667800000 1.5483543489600915
667900000 0.9713811036070346
668000000 0.941270351758688
668100000 0.9776185286576681
668200000 0.9460385113294045
668300000 0.933982397405433
668400000 0.9688785365578055
668500000 0.973401856955423
668600000 0.9894400139146488
668700000 0.9911857902846496
668800000 1.0116895706873417
668900000 0.9917231939757187
669000000 0.9735252181574756
669100000 0.9596593920574724
669200000 0.9863068076607577
669300000 1.062845736545621
669400000 1.100215350495837
669500000 0.9857248565944368
669600000 1.0513423052083222
669700000 0.9992372824781061
669800000 1.1477816221541193
669900000 1.1414934965451384
670000000 1.0686864439863781
670100000 1.0568960962431988
670200000 1.0338930999041198
670300000 1.1233373968067806
670400000 1.0861869136448905
670500000 1.0880593135209116
670600000 1.0429421614425243
670700000 1

692100000 1.836656698527689
692200000 1.7277609969794272
692300000 1.246341020034053
692400000 2.221843279408988
692500000 1.5162819589385115
692600000 1.334287539654712
692700000 2.2435833982788203
692800000 1.1851517487665533
692900000 1.614891352558402
693000000 1.8708570377709295
693100000 1.4591267689867975
693200000 1.7786422778438389
693300000 1.7079903707991733
693400000 1.271366084745854
693500000 1.9443692169868905
693600000 1.3569277424222985
693700000 1.3909290879751128
693800000 1.7066572025565836
693900000 1.7060539374775125
694000000 1.5887307581490098
694100000 1.6521533423890484
694200000 1.5738503123152692
694300000 1.6632084658094606
694400000 1.58988767846846
694500000 1.6706591589611566
694600000 1.2901867063081542
694700000 1.5673538808673122
694800000 1.9446617922081293
694900000 1.468492637446469
695000000 1.4724878851323229
695100000 1.794846432170831
695200000 1.5112839259257813
695300000 1.529066097121744
695400000 1.4257959381242806
695500000 2.0595718649636

713200000 0.10682444139024218
best so far: 0
type: [1, 9, 1, 15, 1] 135
cases of this type: 30375
best so far: 0
type: [1, 9, 3, 1, 5] 135
cases of this type: 50625
best so far: 0
type: [1, 9, 3, 5, 1] 135
cases of this type: 10125
713300000 0.10428146212594509
best so far: 0
type: [1, 9, 5, 1, 3] 135
cases of this type: 18225
best so far: 0
type: [1, 9, 5, 3, 1] 135
cases of this type: 6075
best so far: 0
type: [1, 9, 15, 1, 1] 135
cases of this type: 2025
best so far: 0
type: [1, 15, 1, 1, 9] 135
cases of this type: 98415
713400000 0.10743140678567623
best so far: 0
type: [1, 15, 1, 3, 3] 135
cases of this type: 32805
best so far: 0
type: [1, 15, 1, 9, 1] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 1, 3] 135
cases of this type: 10935
best so far: 0
type: [1, 15, 3, 3, 1] 135
cases of this type: 3645
best so far: 0
type: [1, 15, 9, 1, 1] 135
cases of this type: 1215
best so far: 0
type: [1, 27, 1, 1, 5] 135
cases of this type: 16875
713500000 0.11056654187844199
best

best so far: 0
type: [9, 1, 1, 3, 5] 135
cases of this type: 16875
best so far: 0
type: [9, 1, 1, 5, 3] 135
cases of this type: 10125
best so far: 0
type: [9, 1, 1, 15, 1] 135
cases of this type: 3375
723000000 0.0003253787332097689
best so far: 0
type: [9, 1, 3, 1, 5] 135
cases of this type: 5625
best so far: 0
type: [9, 1, 3, 5, 1] 135
cases of this type: 1125
best so far: 0
type: [9, 1, 5, 1, 3] 135
cases of this type: 2025
best so far: 0
type: [9, 1, 5, 3, 1] 135
cases of this type: 675
best so far: 0
type: [9, 1, 15, 1, 1] 135
cases of this type: 225
best so far: 0
type: [9, 3, 1, 1, 5] 135
cases of this type: 1875
best so far: 0
type: [9, 3, 1, 5, 1] 135
cases of this type: 375
best so far: 0
type: [9, 3, 5, 1, 1] 135
cases of this type: 75
best so far: 0
type: [9, 5, 1, 1, 3] 135
cases of this type: 405
best so far: 0
type: [9, 5, 1, 3, 1] 135
cases of this type: 135
best so far: 0
type: [9, 5, 3, 1, 1] 135
cases of this type: 45
best so far: 0
type: [9, 15, 1, 1, 1] 135
cases o